In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

/Users/tzeng/repos/math_comps


In [2]:
import pickle

# The object code

In [99]:
# Note have to directly paste into notebook

class Tau:
    
    def __init__(self, n):


        self.n = n

        self.G = SymmetricGroup(n)
        self.A = AlternatingGroup(n)

        self.fixed_h = PermutationGroupElement([tuple([i for i in range(1, n+1)])])
        self.ident = self.G.identity()
        self.irr_G = self.G.irreducible_characters()


    
    @staticmethod
    def commutator(x,y):
        return x*y*x^-1*y^-1
    
    @staticmethod
    def ct_to_cs(ct):
        '''
        converts cycle type to cycle string
        '''
        i = 1
        cs = ''

        for n in ct:
            cs += '('
            for _ in range(n):
                cs += f'{i},'
                i+= 1
            
            cs = cs[:-1] + ')'
        
        return cs

    def compute_closed_form(self, tau, model='rand', conj_tau=True):
        '''
        model types: rand, fixed, fixed_conj
        '''

        conj_tau_size = self.G.conjugacy_class(tau).cardinality()

        if model == 'rand':
                prob = sum(chi(tau) / chi(self.ident) for chi in self.irr_G) / factorial(self.n)
        elif model == 'fixed':
            prob = sum(complex(abs(chi(self.fixed_h))^2) * conjugate(chi(tau)) / chi(self.ident) for chi in self.irr_G) / factorial(self.n)
        else: # fixed cong
            prob = sum(complex(abs(chi(self.fixed_h))^2) * chi(tau) / chi(self.ident) for chi in self.irr_G) / factorial(self.n)

        if conj_tau:
            return N(prob * conj_tau_size)
        else:
            return N(prob)


    def compute_brute(self, model='rand'):
        
        ctype_success = {}

        if model=='rand':
            ctype_success = self.__brute_rand()
        elif model == 'fixed':
            ctype_success = self.__brute_fixed()
        else:
            ctype_success = self.__brute_fixed_conj()

        
  

        return ctype_success

    def __brute_rand(self):
        '''
        computes probability for random model (small n)
        '''
        ctype_success = {}

        for h in self.G:
            for v in self.G:

                com = Tau.commutator(h,v).cycle_string()

                ctype_success[com] = ctype_success.get(com,0) + 1
        for com in ctype_success.keys():    
            ctype_success[com] = N(ctype_success[com]/(self.G.cardinality()^2))

        return ctype_success

    def __brute_fixed(self):
        
        ctype_success = {}

        for v in self.G:
            com = Tau.commutator(self.fixed_h,v).cycle_string()#.cycle_type()

            ctype_success[com] = ctype_success.get(com,0) + 1

        for com in ctype_success.keys():    
            ctype_success[com] = N(ctype_success[com]/self.G.cardinality())

        return ctype_success

    def __brute_fixed_conj(self):

        ctype_success = {}

        for h in self.G.conjugacy_class(self.fixed_h):
            for v in self.G:
                com = Tau.commutator(h,v).cycle_string()

                ctype_success[com] = ctype_success.get(com,0) + 1
       


        for com in ctype_success.keys():    
            ctype_success[com] = N(ctype_success[com]/(self.G.cardinality()*self.G.conjugacy_class(self.fixed_h).cardinality()))

        return ctype_success




In [120]:
t = Tau(5)

In [78]:
t.compute_brute(model='rand')

{[1, 1, 1, 1, 1]: 0.0583333333333333,
 [3, 1, 1]: 0.350000000000000,
 [5]: 0.333333333333333,
 [2, 2, 1]: 0.258333333333333}

In [86]:
a = t.compute_brute(model='fixed').keys()

In [121]:
b = t.compute_brute(model='fixed')
c = t.compute_brute(model='rand')
d = t.compute_brute(model='rand_conj')


In [122]:
len(b)

24

In [123]:
len(c)

60

In [111]:
len(d)

2520

In [110]:
len(b)

720

In [135]:
for k in b.keys():
    print(k, b[k])

() 0.0416666666666667
(1,5,2) 0.0416666666666667
(1,5,3) 0.0416666666666667
(1,5,4) 0.0416666666666667
(2,4,3) 0.0416666666666667
(1,5,2,4,3) 0.0416666666666667
(1,5,4,3,2) 0.0416666666666667
(1,5,3,2,4) 0.0416666666666667
(2,5,3) 0.0416666666666667
(1,3)(2,5) 0.0416666666666667
(1,3,2,5,4) 0.0416666666666667
(1,3,2) 0.0416666666666667
(3,5,4) 0.0416666666666667
(1,4,3,5,2) 0.0416666666666667
(1,4)(3,5) 0.0416666666666667
(1,4,3) 0.0416666666666667
(2,5,4) 0.0416666666666667
(1,4)(2,5) 0.0416666666666667
(1,4,2) 0.0416666666666667
(1,4,2,5,3) 0.0416666666666667
(2,4)(3,5) 0.0416666666666667
(1,3,5,2,4) 0.0416666666666667
(1,3)(2,4) 0.0416666666666667
(1,3,5,4,2) 0.0416666666666667


In [87]:
Taus = list(map(PermutationGroupElement, map(Tau.ct_to_cs, a)))

In [14]:
Taus = pickle.load(open('./tau_20.sav', 'rb'))


In [126]:
Taus

[(), (1,2,3), (1,2,3,4,5), (1,2)(3,4)]

In [134]:
for tau in t.G.conjugacy_class(Taus[3]):
    print(tau, t.compute_closed_form(tau, model='fixed_conj', conj_tau=False))



(2,3)(4,5) 0.0138888888888889
(1,2)(3,4) 0.0138888888888889
(1,2)(3,5) 0.0138888888888889
(1,2)(4,5) 0.0138888888888889
(1,3)(4,5) 0.0138888888888889
(1,4)(3,5) 0.0138888888888889
(1,4)(2,5) 0.0138888888888889
(1,3)(2,5) 0.0138888888888889
(1,3)(2,4) 0.0138888888888889
(1,4)(2,3) 0.0138888888888889
(1,5)(2,3) 0.0138888888888889
(1,5)(2,4) 0.0138888888888889
(1,5)(3,4) 0.0138888888888889
(2,5)(3,4) 0.0138888888888889
(2,4)(3,5) 0.0138888888888889


In [76]:
Tau.ct_to_cs(list(a)[2])

'(1,2,3,4,5)'

In [21]:
for tau in Taus:
    print(tau.cycle_type())

[19, 1]
[15, 5]
[14, 6]
[10, 4, 4, 2]
[10, 10]
[13, 3, 2, 2]
[11, 4, 2, 1, 1, 1]
[8, 5, 5, 2]
[11, 6, 2, 1]
[9, 5, 3, 3]
[13, 7]
[14, 3, 2, 1]
[16, 2, 1, 1]
[8, 6, 3, 3]
[6, 6, 4, 2, 1, 1]


In [18]:
for tau in Taus:
    prob = t.compute_closed_form(tau, model='rand')
    print(prob)

0.104691358024691
0.0253992416536299
0.0226755955878344
0.00297592906347518
0.00952492940169316
0.00611296924655288
0.00419665943509088
0.00238108902938643
0.0150644983161956
0.00235791398991967
0.0209317653722839
0.0237082493912893
0.0326733310408184
0.00221016484130151
0.00181532202230302


In [23]:
for tau in Taus:
    prob = t.compute_closed_form(tau, model='fixed')
    print(prob)

0.105263157894737
0.0253984634789588
0.0226751519321179
0.00295945884181178
0.00952375797577036
0.00604664942900237
0.00417637917637918
0.00236866075101369
0.0150625518272577
0.00235594712718896
0.0209317195164166
0.0236928104575163
0.0326797385620915
0.00220840947965131
0.00181492889826223


In [8]:
for tau in Taus:
    print(N(t.G.conjugacy_class(tau).cardinality()/t.A.cardinality()))

0.105263157894737
0.0266666666666667
0.0238095238095238
0.00312500000000000
0.0100000000000000
0.00641025641025641
0.00378787878787879
0.00250000000000000
0.0151515151515152
0.00246913580246914
0.0219780219780220
0.0238095238095238
0.0312500000000000
0.00231481481481481
0.00173611111111111
